# AffirMIDI Piano (ver. 1.0)

## Advaced MIDI Piano affirmator and music generator

***

## Powered by tegridy-tools https://github.com/asigalov61/tegridy-tools

***

### Project Los Angeles

### Tegridy Code 2021

***

# Setup Environment

In [ ]:
#@title Install all dependencies (run only once per session)
%cd /content/
!git clone https://github.com/asigalov61/tegridy-tools
!pip install tqdm

In [ ]:
#@title Import all needed modules
%cd /content/
print('Loading needed modules. Please wait...')
import os
from datetime import datetime
import secrets
import random
import copy
import tqdm
from tqdm import auto

if not os.path.exists('/content/Dataset'):
    os.makedirs('/content/Dataset')

print('Loading TMIDIX module...')
os.chdir('/content/tegridy-tools/tegridy-tools')
import TMIDIX

from pprint import pprint

import numpy as np
import matplotlib.pyplot as plt

from IPython.display import display, Javascript, HTML, Audio

from google.colab import output, drive

os.chdir('/content/')
print('Loading complete. Enjoy! :)')

# (RECOMMENDED) Download ready-to-use MIDI dataset

In [ ]:
#@title Download processed GiantMIDI dataset
%cd /content/
!wget --no-check-certificate -O GiantMIDI.zip "https://onedrive.live.com/download?cid=8A0D502FC99C608F&resid=8A0D502FC99C608F%2118491&authkey=AKrxNM53z9DGX2Y"
!unzip -j GiantMIDI.zip

In [ ]:
#@title Load and prep GiantMIDI dataset
dataset_time_denominator = 10 #@param {type:"slider", min:1, max:20, step:1}
print('=' * 80)
print('Loading GiantMIDI. Please wait a while...')
print('=' * 80)
GiantMIDI = TMIDIX.Tegridy_Any_Pickle_File_Reader('/content/GiantMIDI')
print('=' * 80)

print('Creating TXT and INTs lists...')
print('=' * 80)
TXT, INTs_f = TMIDIX.Optimus_Data2TXT_Converter(GiantMIDI[1], 
                                                dataset_time_denominator=dataset_time_denominator)
print('Writing dataset to files...')
print('=' * 80)
TMIDIX.Tegridy_Any_Pickle_File_Writer(INTs_f, '/content/GiantMIDI-INTs')

print('=' * 80)
with open('/content/GiantMIDI-TXT' + '.txt', 'wb') as f:
  f.write(TXT.encode('utf-8', 'replace'))
  f.close

print('=' * 80)

# (ALTERNATIVE) Use your own MIDI dataset

### NOTES: 

### 1) This will overwrite a previously loaded dataset

### 2) If you are not sure where to start or what settings to select, please use **set** defaults

In [ ]:
#@title Download special Tegridy Piano MIDI dataset (Recommended)

#@markdown Solo Piano

#@markdown Works best stand-alone/as-is for the optimal results
%cd /content/Dataset/

!wget 'https://github.com/asigalov61/Tegridy-MIDI-Dataset/raw/master/Tegridy-Piano-CC-BY-NC-SA.zip'
!unzip -j '/content/Dataset/Tegridy-Piano-CC-BY-NC-SA.zip'
!rm '/content/Dataset/Tegridy-Piano-CC-BY-NC-SA.zip'

%cd /content/

In [ ]:
#@title Process MIDIs to special MIDI dataset with Tegridy MIDI Processor
#@markdown NOTES:

#@markdown 1) Dataset MIDI file names are used as song names. Feel free to change it to anything you like.

#@markdown 2) Best results are achieved with the single-track, single-channel, single-instrument MIDI 0 files with plain English names (avoid special or sys/foreign chars)

#@markdown 3) MIDI Channel = -1 means all MIDI channels except the drums. MIDI Channel = 16 means all channels will be processed. Otherwise, only single indicated MIDI channel will be processed.

desired_dataset_name = "AffirMIDI-Piano-Music-Dataset" #@param {type:"string"}
file_name_to_output_dataset_to = "/content/AffirMIDI-Piano-Music-Dataset" #@param {type:"string"}
desired_MIDI_channel_to_process = 0 #@param {type:"slider", min:-1, max:16, step:1}
sorted_or_random_file_loading_order = False #@param {type:"boolean"}
encode_velocities = False #@param {type:"boolean"}
encode_MIDI_channels = False #@param {type:"boolean"}
add_transposed_dataset_by_this_many_pitches = 0 #@param {type:"slider", min:-12, max:12, step:1}
add_transposed_and_flipped_dataset = False #@param {type:"boolean"}
chordify_input_MIDIs = False #@param {type:"boolean"}
melody_conditioned_chords = False #@param {type:"boolean"}
melody_pitch_baseline = 60 #@param {type:"slider", min:0, max:127, step:1}
time_denominator = 10 #@param {type:"slider", min:1, max:50, step:1}
transform_to_pitch = 0 #@param {type:"slider", min:0, max:127, step:1}
perfect_timings = True #@param {type:"boolean"}
MuseNet_encoding = True #@param {type:"boolean"}
chars_encoding_offset =  33#@param {type:"number"}

print('TMIDI Optimus MIDI Processor')
print('Starting up...')
###########

average_note_pitch = 0
min_note = 127
max_note = 0

files_count = 0

gfiles = 0

chords_list_f = []
melody_list_f = []

chords_list = []
chords_count = 0

melody_chords = []
melody_count = 0

TXT_String = ''

TXT = ''
melody = []
chords = []

###########

print('Loading MIDI files...')
print('This may take a while on a large dataset in particular.')

dataset_addr = "/content/Dataset/"
os.chdir(dataset_addr)
filez = list()
for (dirpath, dirnames, filenames) in os.walk(dataset_addr):
    filez += [os.path.join(dirpath, file) for file in filenames]
print('=' * 70)

if filez == []:
  print('Could not find any MIDI files. Please check Dataset dir...')
  print('=' * 70)

if sorted_or_random_file_loading_order:
  print('Sorting files...')
  filez.sort()
  print('Done!')
  print('=' * 70)

# Stamping the dataset info
print('Stamping the dataset info...')

TXT_String += 'DATASET=' + str(desired_dataset_name) + chr(10)
TXT_String += 'CREATED_ON=' + str(datetime.now()).replace(' ', '-').replace(':', '-').replace('.', '-') + chr(10)

TXT_String += 'CHARS_ENCODING_OFFSET=' + str(chars_encoding_offset) + chr(10)
TXT_String += 'TIME_DENOMINATOR=' + str(time_denominator) + chr(10)
TXT_String += 'TRANSFORM=' + str(transform_to_pitch) + chr(10)
TXT_String += 'PERFECT_TIMINGS=' + str(perfect_timings) + chr(10)
TXT_String += 'MUSENET_ENCODING=' + str(MuseNet_encoding) + chr(10)
TXT_String += 'TRANSPOSED_BY=' + str(add_transposed_dataset_by_this_many_pitches) + chr(10)
TXT_String += 'TRANSPOSED_AND_FLIPPED=' + str(add_transposed_and_flipped_dataset) + chr(10)

TXT_String += 'LEGEND=STA-DUR-PTC'
if encode_velocities:
  TXT_String += '-VEL'
if encode_MIDI_channels:
  TXT_String += '-CHA'
TXT_String += chr(10)

print('Processing MIDI files. Please wait...')
for f in tqdm.auto.tqdm(filez):
  try:
    fn = os.path.basename(f)
    fn1 = fn.split('.')[0]

    files_count += 1
    TXT, melody, chords, bass_melody, karaokez, INTS, aux1, aux2 = TMIDIX.Optimus_MIDI_TXT_Processor(f, chordify_TXT=chordify_input_MIDIs, output_MIDI_channels=encode_MIDI_channels, char_offset=chars_encoding_offset, dataset_MIDI_events_time_denominator=time_denominator, output_velocity=encode_velocities, MIDI_channel=desired_MIDI_channel_to_process, MIDI_patch=range(0, 127), melody_conditioned_encoding=melody_conditioned_chords, melody_pitch_baseline=melody_pitch_baseline, perfect_timings=perfect_timings, musenet_encoding=MuseNet_encoding, transform=transform_to_pitch)
    TXT_String += TXT
    melody_list_f += melody
    chords_list_f.append(chords)
    gfiles += 1

    if add_transposed_dataset_by_this_many_pitches != 0:

      TXT, melody, chords, bass_melody, karaokez, INTS, aux1, aux2 = TMIDIX.Optimus_MIDI_TXT_Processor(f, chordify_TXT=chordify_input_MIDIs, output_MIDI_channels=encode_MIDI_channels, char_offset=chars_encoding_offset, dataset_MIDI_events_time_denominator=time_denominator, output_velocity=encode_velocities, MIDI_channel=desired_MIDI_channel_to_process, transpose_by=add_transposed_dataset_by_this_many_pitches, MIDI_patch=range(0, 127), melody_conditioned_encoding=melody_conditioned_chords, melody_pitch_baseline=melody_pitch_baseline, perfect_timings=perfect_timings, musenet_encoding=MuseNet_encoding, transform=transform_to_pitch)
      TXT_String += TXT
      melody_list_f += melody
      chords_list_f.append(chords)
      gfiles += 1

    if add_transposed_and_flipped_dataset == True:

      TXT, melody, chords, bass_melody, karaokez, INTS, aux1, aux2 = TMIDIX.Optimus_MIDI_TXT_Processor(f, chordify_TXT=chordify_input_MIDIs, output_MIDI_channels=encode_MIDI_channels, char_offset=chars_encoding_offset, dataset_MIDI_events_time_denominator=time_denominator, output_velocity=encode_velocities, MIDI_channel=desired_MIDI_channel_to_process, transpose_by=-12, MIDI_patch=range(0, 127), flip=True, melody_conditioned_encoding=melody_conditioned_chords, melody_pitch_baseline=melody_pitch_baseline, perfect_timings=perfect_timings, musenet_encoding=MuseNet_encoding, transform=transform_to_pitch)
      TXT_String += TXT
      melody_list_f += melody
      chords_list_f.append(chords)
      gfiles += 1

  except KeyboardInterrupt:
    print('Saving current progress and quitting...')
    break  
  
  except:
    print('Bad MIDI:', f)
    continue

TXT_String += 'TOTAL_SONGS_IN_DATASET=' + str(gfiles)

try:
  print('Task complete :)')
  print('==================================================')
  if add_transposed_dataset_by_this_many_pitches != 0:
    print('NOTE: Transposed dataset was added per users request.')
    print('==================================================')
  if add_transposed_and_flipped_dataset == True:
    print('NOTE: Flipped dataset was added per users request.')  
    print('==================================================')
  print('Number of processed dataset MIDI files:', files_count)
  print('Number of MIDI chords recorded:', len(chords_list_f))
  print('First chord event:', chords_list_f[0], 'Last chord event:', chords_list_f[-1]) 
  print('Number of recorded melody events:', len(melody_list_f))
  print('First melody event:', melody_list_f[0], 'Last Melody event:', melody_list_f[-1])
  print('Total number of MIDI events recorded:', len(chords_list_f) + len(melody_list_f))
  print('==================================================')

  # Writing dataset to TXT file
  with open(file_name_to_output_dataset_to + '.txt', 'wb') as f:
    f.write(TXT_String.encode('utf-8', 'replace'))
    f.close

  # Dataset
  MusicDataset = [chords_list_f, melody_list_f]
  print('Creating TXT and INTs lists...')
  print('=' * 70)
  TXT, INTs_f = TMIDIX.Optimus_Data2TXT_Converter(chords_list_f, 
                                                  dataset_time_denominator=time_denominator)
  print('Writing dataset to files...')
  print('=' * 70)
  TMIDIX.Tegridy_Any_Pickle_File_Writer(INTs_f, '/content/Custom-INTs')

  with open('/content/CustomMIDI-TXT' + '.txt', 'wb') as f:
    f.write(TXT.encode('utf-8', 'replace'))
    f.close

  print('=' * 70)
  # Writing dataset to pickle file
  TMIDIX.Tegridy_Any_Pickle_File_Writer(MusicDataset, file_name_to_output_dataset_to)

except:
  print('=' * 70)
  print('IO Error!')
  print('Please check that Dataset dir is not empty/check other IO code.')
  print('=' * 70)
  print('Shutting down...')
  print('=' * 70)

# Affirm your music

In [ ]:
#@title Load your source MIDI file here

#@markdown NOTES:

#@markdown For now, the implementation is hard-coded to a Piano only

#@markdown Best results are achieved with Channel 0, MIDI0, Piano MIDIs.

full_path_to_source_MIDI_file = "/content/tegridy-tools/tegridy-tools/seed2.mid" #@param {type:"string"}

dataset_MIDI_events_time_denominator = 10 #@param {type:"slider", min:1, max:20, step:1}

print('=' * 80)
print('Loading your MIDI...')
dat = TMIDIX.Optimus_MIDI_TXT_Processor(full_path_to_source_MIDI_file, 
                                        dataset_MIDI_events_time_denominator=10,
                                        MIDI_channel=-1,
                                        MIDI_patch=range(0, 127))

print('=' * 80)
print('Processing your MIDI...')

INTs_s = []
time = 0
dur = 0
ptc = 0

pe = dat[2][0]

for d in dat[2]:
  time = abs(d[1]-pe[1])
  dur = d[2]
  ptc = d[4]

  INTs_s.append([time, dur, ptc])
  
  pe = copy.deepcopy(d)

print('=' * 80)
print('Composition has', len(dat[2]), 'notes')

print('=' * 80)
print('Done! Enjoy! :)')

print('=' * 80)

In [ ]:
#@title Affirm your MIDI here
sample_this_many_notes_from_start = 100 #@param {type:"slider", min:50, max:600, step:50}
reaffirm_composition_on_each_run = True #@param {type:"boolean"}

print('=' * 80)
print('AffirMIDI Piano Generator')
print('=' * 80)
print('Starting up...')
print('=' * 80)

song = []
o = 0
s = 0

idx_avg = 0
idx_cnt = 0

idx_list = []

for i in tqdm.tqdm(INTs_s[:sample_this_many_notes_from_start]):
  try:
    idx = TMIDIX.Tegridy_FastSearch(i, INTs_f, randomize=reaffirm_composition_on_each_run)
    
    idx_avg += idx
    idx_cnt += 1
    idx_list.append(idx)
    
    if idx < 0:
      song.append(i)
      o += 1
    
    else:
      ints = INTs_f[idx]
      song.append(ints)
      s += 1
    
  except KeyboardInterrupt:
    break
  
  except:
    print('Error!')
    break

print(chr(10))
print('=' * 80)
print('Original/Affirmed notes count:', o, '/', s)
print('Average distance between notes:', int(idx_avg/idx_cnt), 'dataset notes')

print('=' * 80)
print('Printing compositions dataset indexes list...')
print('=' * 80)
pprint(idx_list, compact=True)

print('=' * 80)
print('Converting composition...')
song_f = []

time = 0
for s in song:
  if len(s) != 0:
    time += s[0]
    song_f.append(['note', time * 10, s[1] * 10, 0, s[2], s[2]+15])
     
print('=' * 80)
print('Writing to TXT and MIDI...')

output_signature = 'AffirMIDI Piano'

# Stuff for datetime stamp
filename = '/content/AffirMIDI-Composition-' + 'generated-on-' 
fname = TMIDIX.Tegridy_File_Time_Stamp(filename)

print('Saving to', str(fname + '.txt'))
with open(fname + '.txt', "w") as text_file:
    print('AffirMIDI Composition', file=text_file)
    print('GiantMIDI Indexes', file=text_file)
    print('\n'.join(str(y) for y in idx_list), file=text_file)
print('=' * 80)

print('Downloading TXT file...')
from google.colab import files
files.download(fname + '.txt')

detailed_stats = TMIDIX.Tegridy_SONG_to_MIDI_Converter(song_f,
                                                      output_signature = output_signature,  
                                                      output_file_name = fname, 
                                                      track_name='Composition # ' + str(idx_list[0]), 
                                                      number_of_ticks_per_quarter=500)

print('=' * 80)
print('Downloading your composition now...')
from google.colab import files
files.download(fname + '.mid')

print('=' * 80)
print('Detailed MIDI stats:')
detailed_stats

# Plot and listen to the last output

In [ ]:
#@title Install prerequisites
!apt install fluidsynth #Pip does not work for some reason. Only apt works
!pip install midi2audio
!pip install pretty_midi

In [ ]:
#@title Plot and listen to the last generated composition
#@markdown NOTE: May be very slow with the long compositions
from midi2audio import FluidSynth
from IPython.display import display, Javascript, HTML, Audio
import pretty_midi
import librosa.display
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import numpy as np

print('Synthesizing the last output MIDI... ')
# fname = '/content/Endless-Piano-Music-Composition'

fn = os.path.basename(fname + '.mid')
fn1 = fn.split('.')[0]

print('Plotting the composition. Please wait...')

pm = pretty_midi.PrettyMIDI(fname + '.mid')

# Retrieve piano roll of the MIDI file
piano_roll = pm.get_piano_roll()

plt.figure(figsize=(14, 5))
librosa.display.specshow(piano_roll, x_axis='time', y_axis='cqt_note', fmin=1, hop_length=160, sr=16000, cmap=plt.cm.hot)
plt.title(fn1)

FluidSynth("/usr/share/sounds/sf2/FluidR3_GM.sf2", 16000).midi_to_audio(str(fname + '.mid'), str(fname + '.wav'))
Audio(str(fname + '.wav'), rate=16000)

# Congrats! You did it! :)